In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model

In [30]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
train_data = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

### 3. Next, from Module 2 (Multiple Regression), copy and paste the ‘get_numpy_data’ function (or equivalent) that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function returns a ‘feature_matrix’ (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ which is an array of the values of the output in the data set (e.g. ‘price’).

In [3]:
def get_numpy_data(df, features, output):
    df['constant'] = 1
    features = ['constant'] + features
    feature_matrix = df[features].values
    output_array = df['price'].values
    return feature_matrix, output_array

### 4. Similarly, copy and paste the ‘predict_output’ function (or equivalent) from Module 2. This function accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and return a 1D array ‘predictions’.

In [4]:
def predict_output(features_matrix,weights):
    predictions=np.dot(features_matrix,weights)
    return predictions

### 5. In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

### 6. Write a short function called ‘normalize_features(feature_matrix)’, which normalizes columns of a given feature matrix. The function should return a pair ‘(normalized_features, norms)’, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.

In [35]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/norms
    return (normalized_features, norms)

In [37]:
normalize_features([[3,5,8],[4,12,15]])

(array([[0.6       , 0.38461538, 0.47058824],
        [0.8       , 0.92307692, 0.88235294]]),
 array([ 5., 13., 17.]))

# Review of Coordinate Descent

# 7. We seek to obtain a sparse set of weights by minimizing the LASSO cost function

    SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).

(By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for

    argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
    
where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.

    Pick a coordinate i
    Compute w[i] that minimizes the LASSO cost function
    Repeat the two steps for all coordinates, multiple times

# 8. For this assignment, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:

           ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
    w[i] = ├  0                     if -lambda/2 <= ro[i] <= lambda/2
           └ (ro[i] - lambda/2)    if ro[i] > lambda/2
           
where

    ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
    
Note that we do not regularize the weight of the constant feature (intercept) w[0], so, for this weight, the update is simply:

    w[0] = ro[i]

# Effect of L1 penalty

# 9. Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

    First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
    Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
    Set initial weights to [1,4,1].
    Make predictions with feature matrix and initial weights.
    Compute values of ro[i], where

In [7]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [8]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [9]:
weights = np.array([1.,4.,1.])

In [10]:
prediction = predict_output(simple_feature_matrix, weights)

Compute values of ro[i], where

    ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]

In [11]:
#calculate ro_i

i = 1
feature_i = simple_feature_matrix[:,i]
w = weights
ro_1 = (feature_i * (output - prediction + w[i]* feature_i)).sum()
ro_1

87939470.82325175

In [12]:
# calculate ro[2]

i = 2
feature_i = simple_feature_matrix[:,i]
w = weights
ro_2 = (feature_i * (output - prediction + w[i]* feature_i)).sum()
ro_2

80966698.66623947

In [13]:
2*ro_1

175878941.6465035

In [14]:
2*ro_2

161933397.33247894


# the combined range of l1_penalty: l1_penalty>2*ro_1

Answer: l1_penalty>175878941.64650351

So we can say that ro[i] quantifies the significance of the i-th feature: the larger ro[i] is, the more likely it is for the i-th feature to be retained.
Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.


In [15]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] *(output - prediction + weights[i]*feature_matrix[:,i])).sum()
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [16]:
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


 
For each iteration:

    1.As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
    2.After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

In [17]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converged = False 
    weights = initial_weights.copy()
    print(weights) 
    while not converged:
        
        changes = []
        for i in range(len(weights)):
            
            old_weights_i = weights[i] 
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            
            
            changes_i = np.abs(weights[i]-old_weights_i)
            changes.append(changes_i) 
            
        if max(changes)<tolerance:
            converged = True
        else:
            converged = False
    return weights



Using the following parameters, learn the weights on the sales dataset.


In [18]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = [0.0,0.0,0.0]
l1_penalty = 1e7
tolerance = 5e5

In [19]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix)

In [20]:
normalized_simple_feature_matrix

array([[0.00680209, 0.00353021, 0.00583571],
       [0.00680209, 0.00768869, 0.00583571],
       [0.00680209, 0.00230361, 0.00389048],
       ...,
       [0.00680209, 0.00305154, 0.00389048],
       [0.00680209, 0.00478673, 0.00583571],
       [0.00680209, 0.00305154, 0.00389048]])

In [23]:
import time

start = time.time()
new_weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, 
                                                output,initial_weights, l1_penalty, 
                                                tolerance)
end = time.time()
print(end - start)

[0.0, 0.0, 0.0]
0.02987504005432129


In [24]:
new_weights

[23964081.329198375, 61017488.64199494, 0.0]

In [25]:
prediction = predict_output(normalized_simple_feature_matrix, new_weights)
error = prediction - output
RSS = np.sum(error*error)
print ('Answer: RSS is', RSS)

Answer: RSS is 1652782810029288.8
